In [1]:
import gzip
import struct
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold, cross_val_score

1. Перших 2 байти завжди порожні тому ми їх пропускаємо зчитуючи >H тип unsigned short який займає 2 байти

2. 3-ій байт вказує на тип даних
  * 0x08: unsigned byte 
  * 0x09: signed byte 
  * 0x0B: short (2 bytes) 
  * 0x0C: int (4 bytes) 
  * 0x0D: float (4 bytes) 
  * 0x0E: double (8 bytes)
  
3. Наступний байт вказує кількість вимірів матриці

4. Далі послідовно для кожного виміру вказано його розмір як 4-байтове ціле число. Тобто, починаючи з п’ятого байту, три групи по 4 байти вказують на розмірність тривимірної матриці, а саме

In [2]:
print('Labels shape')

with gzip.open('dataset/train-labels-idx1-ubyte.gz', 'rb') as f:
    print(struct.unpack('>HBB', f.read(4)))
    print(struct.unpack('>I', f.read(4)))

print('\nImages shape')
with gzip.open('dataset/train-images-idx3-ubyte.gz', 'rb') as f:
    print(struct.unpack('>HBB', f.read(4)))
    print(struct.unpack('>III', f.read(12)))
    
    

Labels shape
(0, 8, 1)
(60000,)

Images shape
(0, 8, 3)
(60000, 28, 28)


In [3]:
# script for reading MNIST file format
def read_idx(filename):
    with gzip.open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [4]:
raw_train = read_idx('dataset/train-images-idx3-ubyte.gz')
X_train = raw_train.reshape(60000, 28 * 28)
y_train = read_idx('dataset/train-labels-idx1-ubyte.gz')

raw_test = read_idx('dataset/t10k-images-idx3-ubyte.gz')
X_test = raw_test.reshape(10000, 28 * 28)
y_test = read_idx('dataset/t10k-labels-idx1-ubyte.gz')

In [5]:
import pandas as pd

def get_subset(X, y, n):
    df = pd.DataFrame(data=X)
    df['label'] = y
    df = df.groupby('label').apply(pd.DataFrame.sample, n=n).reset_index(drop=True)
    
    return df.iloc[:, :-1], df.label

In [6]:
X_subset, y_subset = get_subset(X_train, y_train,100)

In [7]:
# kf = KFold(n_splits=5, random_state=42, shuffle=True)

# шукаємо найкращий пареметр для knn 
C = [1, 10, 20]
scores = []

for c in C:
    svc = LinearSVC(C=c, verbose=True)
    score = cross_val_score(svc, X_subset, y_subset, cv=5, scoring='accuracy').mean()
    scores.append(score)
    print(c, scores[-1])

# plt.plot(k_range, k_scores)
# plt.xlabel("Value of K for KNN")
# plt.ylabel("Cross Validation Accuracy")

[LibLinear]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

...

[LibLinear]10 0.8379999999999999
[LibLinear][LibLinear]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear][LibLinear]20 0.8379999999999999


In [ ]:
# save classifier
from sklearn.externals import joblib
joblib.dump(knn, 'flaskr/classifiers/svm.pkl') 

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = svn.predict(scaler.transform(X_test))
accuracy_score(y_test, y_pred)